## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
from sklearn.model_selection import train_test_split

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


## Reading the Dataset

In [2]:
dataset = pd.read_csv('Sentiment.csv')

In [3]:
dataset

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0000,yes,1.0000,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0000,yes,1.0000,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0000,yes,1.0000,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13866,13867,No candidate mentioned,1.0000,yes,1.0000,Negative,0.7991,Abortion,0.6014,No candidate mentioned,...,yes,7,Negative,Abortion\nWomen's Issues (not abortion though),RT @cappy_yarbrough: Love to see men who will ...,NaN,2015-08-07 09:29:43 -0700,629690895479250944,Como,NaN
13867,13868,Mike Huckabee,0.9611,yes,1.0000,Positive,0.7302,None of the above,0.9229,Mike Huckabee,...,yes,1,NaN,NaN,RT @georgehenryw: Who thought Huckabee exceede...,NaN,2015-08-07 09:25:02 -0700,629689719056568320,USA,NaN
13868,13869,Ted Cruz,1.0000,yes,1.0000,Positive,0.8051,None of the above,0.9647,Ted Cruz,...,yes,67,Positive\nNeutral,NaN,"RT @Lrihendry: #TedCruz As President, I will a...",NaN,2015-08-07 07:19:18 -0700,629658075784282112,NaN,NaN
13869,13870,Donald Trump,1.0000,yes,1.0000,Negative,1.0000,Women's Issues (not abortion though),0.9202,Donald Trump,...,yes,149,NaN,Women's Issues (not abortion though),RT @JRehling: #GOPDebate Donald Trump says tha...,NaN,2015-08-07 09:54:04 -0700,629697023663546368,NaN,NaN


In [4]:
dataset = dataset[['text','sentiment']]

In [5]:
dataset

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


In [6]:
dataset = dataset[dataset['sentiment']!= 'Neutral']

In [7]:
dataset

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


## Data Preprocessing

In [8]:
train,test = train_test_split(dataset,test_size=0.1)

In [19]:
train

,text,sentiment
9720,RT @RWSurferGirl: It is very disappointing tha...,Negative
9202,@therobotjane @ChZuckBean @BillyDenton23 @Shar...,Negative
11130,Ben Carson doesn't talk trash. Mature speaker ...,Positive
1846,I'm still a little sick to my stomach from wat...,Negative
1126,Here's why it was so frustrating to watch the ...,Negative
...,...,...
2171,@RealBenCarson @DrBenCarson2016 #GOPDebate I l...,Positive
10243,"RT @SupermanHotMale: Dear Ted Cruz, what do sc...",Negative
11695,RT @RWSurferGirl: Why doesn't Chris Wallace as...,Negative
2107,"RT @AnnTelnaes: God, I hope the field gets sma...",Negative


In [20]:
test

,text,sentiment
536,Strange. God told me NOT to vote for them. #GO...,Negative
11494,"RT @SupermanHotMale: Hey Chris Wallace, These ...",Negative
5551,"RT @LegalizeNegroes: ""WE DON'T NEED A PRESIDEN...",Negative
864,"I mean, why should it matter if the questions ...",Negative
11166,#GOPDebates Ben Carson take my advice and try ...,Negative
...,...,...
13469,Summary of #GOPDebates: big ol circle jerk shi...,Negative
9446,First debate seems to have had better moderato...,Negative
10965,RT @RWSurferGirl: Fox News is obviously trying...,Negative
5608,"#GOPDebate best one I've ever seen. tactful,...",Positive


In [21]:
train['text'][9720]

'RT @RWSurferGirl: It is very disappointing that Fox News is not conducting a "Fair &amp; Balanced" Debate. 🇺🇸 #GOPDebate  #GOPDebates'

In [28]:
pattern = "(#\w+)|(RT\s@\w+:)|(http.*)|(@\w+)"

In [24]:
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [25]:
def Clean_text(data):
    tweets = []
    sentiments = []
    for index,row in data.iterrows():
        sentence = re.sub(pattern,'',row.text)
        words = [e.lower() for e in sentence.split()]
        words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords.words('english')]
        words = ' '.join(words)
        tweets.append(words)
        sentiments.append(row.sentiment)
    return tweets,sentiments

In [26]:
train_tweets,train_sentiments = Clean_text(train)

In [31]:
final_data = {'tweets':train_tweets,'sentiments':train_sentiments}

In [32]:
processed_data = pd.DataFrame(final_data)

In [33]:
processed_data

,tweets,sentiments
0,"disappointing fox news conducting ""fair &amp; ...",Negative
1,starscream dick cheney,Negative
2,ben carson talk trash. mature speaker truth. m...,Positive
3,i'm still little sick stomach watching gop deb...,Negative
4,here's frustrating watch woman,Negative
...,...,...
9651,loved quote true is!!!,Positive
9652,"dear ted cruz, scripture say lying good presid...",Negative
9653,chris wallace ask politicans finance money com...,Negative
9654,"god, hope field get smaller soon...",Negative


In [34]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
processed_data['sentiments'] = labelencoder.fit_transform(processed_data['sentiments'])

In [35]:
processed_data

,tweets,sentiments
0,"disappointing fox news conducting ""fair &amp; ...",0
1,starscream dick cheney,0
2,ben carson talk trash. mature speaker truth. m...,1
3,i'm still little sick stomach watching gop deb...,0
4,here's frustrating watch woman,0
...,...,...
9651,loved quote true is!!!,1
9652,"dear ted cruz, scripture say lying good presid...",0
9653,chris wallace ask politicans finance money com...,0
9654,"god, hope field get smaller soon...",0


## Converting Words into Vectors

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1,3))
cv.fit(processed_data['tweets'])

CountVectorizer(ngram_range=(1, 3))

In [37]:
X_train = cv.transform(processed_data['tweets'])

In [38]:
print(X_train.shape)

(9656, 81797)


In [44]:
X_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [39]:
target = processed_data['sentiments'].values

In [41]:
target

array([0, 0, 1, ..., 0, 0, 1])

## Sentiment Analysis (Model Building)

In [42]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()

In [45]:
classifier.fit(X_train.toarray(),target)

MultinomialNB()

In [47]:
test_tweets,test_sentiments = Clean_text(test)

In [50]:
data_test = {'tweets':test_tweets,'sentiments':test_sentiments}
final_test_data = pd.DataFrame(data_test)

In [52]:
X_test = cv.transform(final_test_data['tweets'])

In [55]:
X_test.shape

(1073, 81797)

In [56]:
y_pred = classifier.predict(X_test.toarray())

In [60]:
final_test_data['sentiments'] = labelencoder.fit_transform(final_test_data['sentiments'])

In [61]:
final_test_data


,tweets,sentiments
0,strange. god told vote them.,0
1,"hey chris wallace, guy better plan president o...",0
2,"""we need president cant say extremist islamic ...",0
3,"mean, matter question unfair? bother answering...",0
4,ben carson take advice try surgeon general,0
...,...,...
1068,summary : big ol circle jerk shitting democrats.,0
1069,first debate seems better moderator carly. sor...,0
1070,fox news obviously trying influence makeup rep...,0
1071,"best one i've ever seen. tactful, informative,...",1


In [62]:
actual_values = final_test_data['sentiments'].values

In [63]:
actual_values

array([0, 0, 0, ..., 0, 1, 0])

In [64]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred,actual_values))

0.8527493010251631


In [65]:
final_test_data

,tweets,sentiments
0,strange. god told vote them.,0
1,"hey chris wallace, guy better plan president o...",0
2,"""we need president cant say extremist islamic ...",0
3,"mean, matter question unfair? bother answering...",0
4,ben carson take advice try surgeon general,0
...,...,...
1068,summary : big ol circle jerk shitting democrats.,0
1069,first debate seems better moderator carly. sor...,0
1070,fox news obviously trying influence makeup rep...,0
1071,"best one i've ever seen. tactful, informative,...",1


In [68]:
X_test.toarray()[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [70]:
classifier.predict(X_test.toarray()[[0]])

array([0])